In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%%time

# 데이터 경로 맞춰주세요!
TRAIN_DATA_PATH = './riiid_data/train.csv'
df_train = pd.read_csv(TRAIN_DATA_PATH)
df_train.head()

Wall time: 3min 8s


,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


In [ ]:
# 데이터 경로 맞춰주세요!
QUESTION_DATA_PATH = './riiid_data/questions.csv'
df_question = pd.read_csv(QUESTION_DATA_PATH)
df_question.head()

In [ ]:
# 필요없는 데이터 drop하기
df_train = df_train.drop(['row_id', 'content_type_id', 'user_answer', 'prior_question_elapsed_time', 'prior_question_had_explanation'], axis=1)

In [ ]:
# 메모리 부족할 경우 아래 코드 사용해서 저장 후 다시 불러오기
# df_train.to_csv('riiid.csv', index=False, header=True)

In [ ]:
# df_train = pd.read_csv('riiid.csv')

In [3]:
# lecture 관련 data drop하기
print('before filtering: ', len(df_train['answered_correctly']))
df_train = df_train[df_train['answered_correctly'] != -1]
print('after filtering: ', len(df_train['answered_correctly']))

before filtering:  101230332
after filtering:  99271300


In [ ]:
# train과 qustion 관련 table 합치기
df_train = pd.merge(df_train, df_question, left_on="content_id", right_on="question_id", how='left')
df_train.head()

In [ ]:
# 중복되는 column drop
df_train = df_train.drop('content_id', axis=1)

In [ ]:
# 메모리 부족할 경우 저장 후 다시 불러오기
# df_train.to_csv('riiid.csv', index=False, header=True)

In [ ]:
# df_train = pd.read_csv('riiid.csv')

In [ ]:
# 대회 트레인 셋에 맞게 testId 생성
df_train['task_container_id'] = df_train['task_container_id'].astype('str')
df_train['task_container_id'] = df_train['task_container_id'].apply(lambda x: '0' + '00' + ('0' * (4 - len(x))) + x)

df_train['part'] = df_train['part'].astype('str')
df_train['part'] = df_train['part'].apply(lambda x: 'A0' + x)

df_train['testId'] = df_train['part'] + df_train['task_container_id']

In [ ]:
# 메모리 부족할 경우 저장 후 다시 불러오기
# df_train.to_csv('riiid.csv', index=False, header=True)

In [ ]:
# df_train = pd.read_csv('riiid.csv')

In [ ]:
# 필요없는 column 드랍하기
df_train = df_train.drop(['task_container_id', 'part'], axis=1)

In [ ]:
# 대회 train set에 맞게 assessmentItemID 생성
df_train['pre_asses_id'] = df_train['testId'].apply(lambda x: x[0] + x[2] + x[-3:])

In [ ]:
# 메모리 부족할 경우 저장 후 다시 불러오기
# df_train.to_csv('riiid.csv', index=False, header=True)

In [ ]:
# df_train = pd.read_csv('riiid.csv')

In [ ]:
df_train['question_id'] = df_train['question_id'].astype('str')
df_train['question_id'] = df_train['question_id'].apply(lambda x: ('0' * (5 - len(x))) + x)

In [ ]:
# 메모리 부족할 경우 저장 후 다시 불러오기
# df_train.to_csv('riiid.csv', index=False, header=True)

In [ ]:
# df_train = pd.read_csv('riiid.csv')

In [ ]:
df_train['assessmentItemID'] = df_train['pre_asses_id'] + df_train['question_id']

In [ ]:
# 필요 없는 column들 drop하기
df_train = df_train.drop(['question_id', 'pre_asses_id'], axis=1)

In [ ]:
# 메모리 부족할 경우 저장 후 다시 불러오기
# df_train.to_csv('riiid.csv', index=False, header=True)

In [2]:
# df_train = pd.read_csv('riiid.csv')

In [3]:
# 대회 dataset에 맞게 column 이름 변경
df_train = df_train.rename(columns={'timestamp': 'Timestamp', 'user_id': 'userID', 'answered_correctly': 'answerCode'})

,Timestamp,userID,answerCode,tags,testId,assessmentItemID
0,0,115,1,151,A050000001,A500105692
1,56943,115,1,168,A050000002,A500205716
2,118363,115,1,131 149 92,A010000000,A100000128
3,131167,115,1,131 104 81,A010000003,A100307860
4,137965,115,1,131 149 92,A010000004,A100407922


In [4]:
# 대회 트레인 셋에 맞게 KnowledgeTag 생성
df_train['tags'] = df_train['tags'].astype('str')
df_train['KnowledgeTag'] = df_train['tags'].apply(lambda x: x.split(' ')[0])

In [5]:
df_train['KnowledgeTag'] = df_train['KnowledgeTag'].fillna(0)

In [7]:
# Null value가 있는지 체크
df_train.isnull().values.any()

False

In [12]:
# 필요없는 데이터 drop
df_train = df_train.drop('tags', axis=1)

In [14]:
# 최종 데이터 저장
df_train.to_csv('riiid.csv', index=False, header=True)

In [6]:
# 데이터가 너무 큰 경우 대비해서 10% 데이터만 저장
df_riiid_short = df_train[:len(df_riiid)//10]
df_riiid_short.to_csv('riiid_sec1.csv', index=False, header=True)